In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Product Management

In [79]:
responsibility_synonyms = ["WHAT WE'RE LOOKING FOR", 'key responsibilities', "What you bring:",     "Required Competencies & Qualifications:",
    "WHAT WE'RE LOOKING FOR:",
    "We'd love to hear from you if you:","We want you if you are", "About You", 
    "About You:","What You Have",
    "We're Excited About You Because:","Minimum qualifications:","The Ideal Candidate/Bonus Points",
    "Qualifications:",
    "Who Are We Looking For",     "Who You Are:",
    "Required Competencies & Qualifications:","What you need to succeed",
    "WHAT WE'RE LOOKING FOR:",
    "We'd love to hear from you if you:","We prefer a teammate with:",
    "About You:",
    "We're Excited About You Because:","Skills You'll Need to Bring:",
    "Qualifications:",
    "Who Are We Looking For:",
    "basic qualifications:", "We would like to see:", "Preferred Qualifications", "What is required for you to be successful in this role"]

In [35]:
links = pd.read_excel('../data/Job Description Links.xlsx')
links2 = pd.read_csv('../data/url_links_productsthatcount.csv')
links3 = pd.read_csv('../data/url_links_builtinnyc.csv')

In [37]:
links3.drop(columns = 'Unnamed: 0', inplace=True)

In [7]:
links = links.dropna()
#links

In [80]:
#links3

In [81]:
import re

def find_keywords_from_paragraph(div):
    #print(div)

    paragraph_text = div.get_text()
    processed_paragraph_text = re.sub(r'[^a-zA-Z0-9\s]', '', paragraph_text.lower())
    for word in responsibility_synonyms:
        processed_word = re.sub(r'[^a-zA-Z0-9\s]', '', word.lower())
        #print(processed_word)
        if processed_word in processed_paragraph_text:

            # container = div.find_parent(['div', 'header', 'h1', 'h2', 'h3', 'h4', 'h5', 'p'])
            # if container:
                
            content = div.find_next_sibling(["ul", "p"])
            if content:
                print("Found word")
                #print("conten", content)
                if content.name == "ul":
                    terms = [li.get_text() for li in content.find_all("li")]
                    return word, terms
                else:
                    return word, content.get_text()
                #paragraph_element = div.find_next_sibling(["p"])
                # if ul_element:
                #     print("list element")
                #     li_elements = ul_element.find_all("li")
                #     return word, li_elements
                # elif paragraph_element:
                #     print("paragraph element")
                #     return word, paragraph_element.get_text()
            #else:
            #     return word, processed_paragraph_text
        


    

In [82]:
from bs4 import BeautifulSoup, Tag

def go_through_tags(div_elements):
    if div_elements:
        for id, div_element in enumerate(div_elements):

            if div_element and isinstance(div_element, Tag):
                ul_terms = find_keywords_from_paragraph(div_element)
                if ul_terms:
                    return ul_terms
                else:
                    div_within_div = div_element.find_all(["div", "h1", "h2", "h3", "h4", "h5", "p"])
                    #print("Entering this also")
                    for div in div_within_div:
                        result = go_through_tags(div)
                        if result:
                            return result
            #else: None, None, "Div not found"

    # else:
    #     return None, None, "Keyword not found"

In [84]:
from tqdm import tqdm


In [85]:
df = pd.DataFrame(columns = ['url', 'keyword', 'terms', 'error', 'access_date'])

# URL of the page to scrape

url_list = links3['Links'].unique()
    
for idx, url in tqdm(enumerate(url_list)):

    df.at[idx, 'url'] = url

    # Send a GET request to the URL
    try:
        response = requests.get(url)
        if response.status_code == 200:

            # # Parse the HTML content using Beautiful Soup
            soup = BeautifulSoup(response.content, "html.parser")

            div_elements = soup.find_all("div")

            ul = go_through_tags(div_elements)
            df.at[idx, 'terms'] = ul[1]
            df.at[idx, 'keyword'] = ul[0]
            df.at[idx, 'access_date'] = datetime.today()
                

        else:
            df.at[idx, 'error'] = 'Page is unavailable'
    except Exception as e:
        df.at[idx, 'error'] = e


0it [00:00, ?it/s]

4it [01:34, 17.98s/it]

Found word


5it [01:36, 13.47s/it]

Found word


...

Found word


8it [01:41,  6.06s/it]

Found word


9it [01:44,  5.15s/it]

Found word


In [78]:
links3.iloc[7]["Links"]

'https://www.builtinnyc.com/job/product/senior-manager-product-operations/225942'

In [44]:
df.dropna(subset='terms').to_csv('../data/requirements_sheet_jd_buuiltinny.csv')